# Loading And Fetching Data From Reddit PRAW

In [ ]:
SECRET_KEY = 'LyyFU_r17F6s1i0ajYI2dxoSi2dOtw'
CLIENT_ID = 'SMi4R1E-3TeMoXqqwPNwUg'
USER_NAME = 'Weary-Tooth7440'

In [2]:
%pip install praw

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [8]:
import praw

# Initialize Reddit instance
reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_KEY,
                     user_agent=USER_NAME)

In [9]:
# Subreddit to scrape
subreddit = reddit.subreddit('stocks')

# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)
 
# Display the title of the Subreddit
print("Title:", subreddit.title)
 
# Display the description of the Subreddit
print("Description:", subreddit.description)

Display Name: stocks
Title: Stocks - Investing and trading for all
Description: Almost any post related to stocks is welcome; please read the rules below:

[**If you're new here**](https://www.reddit.com/r/stocks/comments/4x1419/if_youre_new_here_read_this_post_first/)

##Resources

* [Wiki for new investors](https://www.reddit.com/r/stocks/wiki/index)

* [Pattern day trading](https://www.reddit.com/r/stocks/wiki/pdtrules)

* [Earnings calendar](https://finance.yahoo.com/calendar/earnings/)

##Karma requirements

[Click here to find how many days old your account needs to be and how much karma you need](https://www.reddit.com/r/stocks/wiki/karma) before you can comment or post to r/Stocks.

##Rules [(in depth rules wiki here)](https://www.reddit.com/r/stocks/wiki/rules)

1. Disclose any related open positions when discussing a particular stock or financial instrument.

2. Spam, ads, solicitations (including referral links), and self-promotion posts or comments will be removed and you m

In [22]:
user_name = "AutoModerator"
redditor = reddit.redditor(user_name)

data = []

for post in subreddit.search('daily discussion', sort='new', time_filter='week'):
      if post.num_comments > 0:
        # Scraping comments for each post
        post.comments.replace_more(limit= 5)
        for comment in post.comments.list():
            data.append({
                'id': post.id + '_' +  comment.id ,
               'Author': comment.author.name if comment.author else 'Unknown',
                'Timestamp': pd.to_datetime(comment.created_utc, unit='s'),
                'Text': comment.body,
                'Score': comment.score,
                'Post_url':post.url,
            })

In [ ]:
# Create pandas DataFrame for posts and comments
df = pd.DataFrame(data)

# Cleaning The Data

In [ ]:
def DropDeletedComment(data):
  # Check for rows where the 'Text' column contains '[deleted]'
  deleted_rows = data[data['Text'].str.contains('\[deleted\]', na=False)]
  
  # Count the number of deleted rows
  num_deleted = len(deleted_rows)
  
  if (num_deleted) > 1:
    #Dropping the text with [deleted]
    data = data[~data['Text'].str.contains('\[deleted\]', na=False)]
    data = data.reset_index(drop=True)
  
  return data

In [ ]:
df = DropDeletedComment(df)

# Preprocessing Comment

In [28]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Jamie/nltk_data'
    - 'c:\\Python311\\nltk_data'
    - 'c:\\Python311\\share\\nltk_data'
    - 'c:\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\Jamie\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
def text_preprocessing(txt):
    # Remove non-word characters and lowercase the text
    txt = re.sub(r'\W+', ' ', txt)
    txt = txt.lower()

    # Tokenize the text
    word_tokens = word_tokenize(txt)

    # Remove stop words
    filtered_words = [w for w in word_tokens if w not in stop_words]

    # Stem or Lemmatize each word
    stemmed_words = [stemmer.stem(w) for w in filtered_words]
    lemmatized_words = [lemmatizer.lemmatize(w) for w in stemmed_words]

    # Join the words back into a single string
    return ' '.join(lemmatized_words)

# Apply the preprocessing function to the DataFrame
df['original_text'] = df['Text']
df['Text'] = df['Text'].apply(text_preprocessing)

# Visualization

In [ ]:
#Visualizing most frequent words
from nltk.probability import FreqDist

# Extracts words into list and count frequency
all_words = ' '.join([text for text in df['Text']])
all_words = all_words.split()
words_df = FreqDist(all_words)

# Extracting words and frequency from words_df object
words_df = pd.DataFrame({'word':list(words_df.keys()), 'count':list(words_df.values())})

# Subsets top 30 words by frequency
words_df = words_df.nlargest(columns="count", n = 30)

words_df.sort_values('count', inplace = True)

# Plotting 30 frequent words
plt.figure(figsize=(20,10))
plt.title("Top 50 Frequent Word")
ax = plt.barh(words_df['word'], width = words_df['count'])
plt.show()

# Training/Predicting

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def get_sentiment(text):
    # Tokenize the input text
    encoded_input = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)  # Pad and truncate to 3625 tokens)
    
    # Get model output
    output = model(**encoded_input)
    
    # Calculate softmax scores
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # Get the predicted label and its score
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    predicted_label = config.id2label[ranking[0]]
    predicted_score = scores[ranking[0]]

    return predicted_label, predicted_score

In [ ]:
# Sort the DataFrame by timestamp in descending order
df = df.sort_values(by='Timestamp', ascending=False)

# Get the latest 1000 rows
latest_df = df.head(10)

results = []
for index, row in latest_df.iterrows():
    text = row['original_text']
    timestamp = row['Timestamp']
    predicted_label, predicted_score = get_sentiment(text)
    results.append([text, timestamp, predicted_label, predicted_score])

    # Debugging information (optional)
    print(f"Text: {text}")
    print(f"Timestamp: {timestamp}")
    print(f"Predicted Label: {predicted_label}")
    print(f"Predicted Score: {predicted_score}\n")

# Create a Pandas DataFrame from the results
results_df = pd.DataFrame(results, columns=['Original Text', 'Timestamp', 'Sentiment', 'Score'])

results_df